## Week 2 Day 3 - MAiljet replacemnt of sendgrid

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

False

In [3]:
# mailjet replacement code
!pip install mailjet_rest
from mailjet_rest import Client
load_dotenv(override=True)

if not os.getenv("OPENAI_API_KEY"):
    print("Error: OPENAI_API_KEY not found!")
    print("Please set your OpenAI API key in a .env file")
else:
    print("Mailjet API key found! Ready to generate mail.")

if not os.getenv("MJ_APIKEY_PUBLIC"):
    print("Error: MJ_APIKEY_PUBLIC not found!")
    print("Please set your mj api key in a .env file")
else:
    api_key = os.environ["MJ_APIKEY_PUBLIC"]
    print(f"API key {api_key} found! Ready to generate mail`.")

if not os.getenv("MJ_APIKEY_PRIVATE"):
    print("Error: MJ_APIKEY_PRIVATE not found!")
    print("Please set your mj api key in a .env file")
else:
    api_secret = os.environ["MJ_APIKEY_PRIVATE"]
    print(f"API decret {api_secret} key found! Ready to generate images.")
# api_secret = os.environ["MJ_APIKEY_PRIVATE"]
from_email=os.environ["FROM_EMAIL"]
from_email="abushy@abushy.com"
# to_email=os.environ["TO_EMAIL"]
to_email="andy.bush.nz@icloud.com"
print(f"from email {from_email} , to email {to_email} ")


Mailjet API key found! Ready to generate mail.
API key 822b05cca548823d0a6b70142cb59673 found! Ready to generate mail`.
API decret 184e961de67cccd9efcd2e1e71033efc key found! Ready to generate images.
from email abushy@abushy.com , to email andy.bush.nz@icloud.com 


In [4]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key exists and begins ecd
Groq API Key exists and begins gsk_


In [5]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [6]:

import requests
pushover_user = os.getenv("PUSHOVER_USER")
# pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_token = "attfgwkdm6c9ssw9jrtqss7zupuvhc"
pushover_url = "https://api.pushover.net/1/messages.json"

def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    print(payload)
    response=requests.post(pushover_url, data=payload)
    print(response)

push("pushover function test")

Push: pushover function test
{'user': 'uf2h4m7izyivoc7xbawznigjdi2pk1', 'token': 'attfgwkdm6c9ssw9jrtqss7zupuvhc', 'message': 'pushover function test'}
<Response [200]>


### It's easy to use any models with OpenAI compatible endpoints

In [7]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [8]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
open_ai_client = AsyncOpenAI(api_key=openai_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)
open_ai_model = OpenAIChatCompletionsModel(model="gpt-4o-mini", openai_client=open_ai_client)

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

instructions4 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3 =  Agent(name="Llama3.3 Sales Agent", instructions=instructions3, model=llama3_3_model)
sales_agent4 =  Agent(name="OpenAI Sales Agent", instructions=instructions4, model=open_ai_model)

# sales_picker = Agent(name="Sales Picker", instructions=instructions5, model=open_ai_model)

In [9]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)
tool4 = sales_agent4.as_tool(tool_name="sales_agent3", tool_description=description)

In [10]:
# modify send email

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    #Send out an ema
    push(f"preparing to email {from_email} , {to_email}, {subject}")
    mailjet = Client(auth=(api_key, api_secret))
    data = {
        "FromEmail": from_email,
        "FromName":"email agent - Sales pircker",
        "Subject": subject,
        "Text-part": "Dear passenger, welcome to Mailjet! May the delivery force be with you!",
        "Html-part": html_body,
        "Recipients": [{"Email": to_email}]
         }
    result = mailjet.send.create(data=data)
    print("mailjet status code ",result.status_code)
    print(result.json())
    push(f"sent email status = {result.status_code} f= ({from_email} ,t = {to_email}")
    push("email end")
    return {"status": "success"}

In [11]:
def send_html_email2(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """

    mailjet = Client(auth=(api_key, api_secret))
    data = {
    "FromEmail": from_email,
    "FromName":"Agent 3 - Sales pircker",
    "Subject": subject,
    # "Text-part": "Dear passenger, welcome to Mailjet! May the delivery force be with you!",
    "Html-part": html_body,
    "Recipients": [{"Email": to_email}]
    }
    result = mailjet.send.create(data=data)
    print(result.status_code)
    print(result.json())
    push(f"sent html_email {from_email} , {to_email} subject = {subject} {html_body}")
    return {"status": "success"}

# send_html_email2("test", "test")


In [12]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [13]:
email_tools = [subject_tool, html_tool, send_html_email]

In [14]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [15]:
# tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
tools = [tool2, tool3]

In [16]:
print(tools)

print(handoffs)

[FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x109e61580>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent3_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x109e619e0>, strict_json_schema=True, is_enabled=True)]
[Agent(name='Email Manager', instructions='You are an email formatter and sender. You receive the body of an email to be sent. You first use the subject_writer tool to w

In [17]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 2 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from the sales manager"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

Push: preparing to email abushy@abushy.com , andy.bush.nz@icloud.com, Streamline Your SOC 2 Compliance Journey with ComplAI
{'user': 'uf2h4m7izyivoc7xbawznigjdi2pk1', 'token': 'attfgwkdm6c9ssw9jrtqss7zupuvhc', 'message': 'preparing to email abushy@abushy.com , andy.bush.nz@icloud.com, Streamline Your SOC 2 Compliance Journey with ComplAI'}
<Response [200]>
mailjet status code  200
{'Sent': [{'Email': 'andy.bush.nz@icloud.com', 'MessageID': 288230407399881532, 'MessageUUID': 'c30b3c46-a9ce-4a51-b388-cf90f0cf76f0'}]}
Push: sent email status = 200 f= (abushy@abushy.com ,t = andy.bush.nz@icloud.com
{'user': 'uf2h4m7izyivoc7xbawznigjdi2pk1', 'token': 'attfgwkdm6c9ssw9jrtqss7zupuvhc', 'message': 'sent email status = 200 f= (abushy@abushy.com ,t = andy.bush.nz@icloud.com'}
<Response [200]>
Push: email end
{'user': 'uf2h4m7izyivoc7xbawznigjdi2pk1', 'token': 'attfgwkdm6c9ssw9jrtqss7zupuvhc', 'message': 'email end'}
<Response [200]>


## Check out the trace:

https://platform.openai.com/traces

## Check out the trace:

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>